In [0]:

# Leer el parquet
df = spark.read.parquet("/Volumes/workspace/raw/hvfhs_volume/fhvhv_tripdata_2025-01.parquet")

# Mostrar las primeras filas
df.show(5)

# Ver el esquema de columnas
df.printSchema()



In [0]:
# Guardar como tabla SQL permanente
df.write.mode("overwrite").saveAsTable("raw.hvfhs_volume")


In [0]:
from pyspark.sql.functions import current_timestamp, col

 
INPUT_PATH = "volume://workspace.raw.hvfhs_volume"

# Leemos los datos tal cual como vienen
df_raw = spark.read.table("raw.hvfhs_volume")

# Agregamos metadata a la ingesta para saber cuando entro el dato al dataLake
df_bronze = (
    df_raw
    .withColumn("ingestion_timestamp", current_timestamp())
    .withColumn("source_file", col("_metadata.file_path"))
)

# Guardamos los datos en la capa bronze con el metodo append
(
    df_bronze.write
    .format("delta")
    .mode("append")
    .saveAsTable("workspace.bronze.hvfhs")
)

print("Ingestión Bronze completada")


In [0]:
%sql
SELECT * 
FROM workspace.bronze.hvfhs
LIMIT 10;
